# Torch
## Check GPU

In [1]:
import torch
import numpy as np
import sys
sys.path.append('..')

from torchlib.utils import list_device,set_device

# S1: check GPU
#list_device()

# S2: default parameters
set_device(1)
np.set_printoptions(precision = 2)
torch.set_default_dtype(torch.float32)
torch.set_printoptions(precision=4)
torch.backends.cudnn.benchmark = True
torch.set_printoptions(sci_mode=False)

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


Using Device 1 : TITAN Xp


# Set Arguments

In [2]:
import argparse
import sys
import os
import time
import pickle

parser = argparse.ArgumentParser()

'''Training Parameters'''
parser.add_argument('--batch_size', type=int, default=5000, help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=200, help='number of epochs')
parser.add_argument('--grad_clip', type=float, default=5., help='clip gradients at this value')
parser.add_argument('--learning_rate', type=float, default=0.001, help='learning rate')
parser.add_argument('--learning_rate_clip', type=float, default=0.0000001, help='learning rate clip')
parser.add_argument('--decay_rate', type=float, default=.95, help='decay rate for rmsprop')
parser.add_argument('--weight_decay', type=float, default=.0001, help='decay rate for rmsprop')
parser.add_argument('--batch_norm_decay', type=float, default=.999, help='decay rate for rmsprop')
parser.add_argument('--keep_prob', type=float, default=1.0, help='dropout keep probability')
parser.add_argument('--lamda_weights', type=float, default=0.01, help='weight of rotation error')
parser.add_argument('--data_argumentation', type=bool, default=True, help='whether do data argument')
parser.add_argument('--is_normalization', type=bool, default=True, help='whether do data normalization')
parser.add_argument('--target_image_size', default=[300, 300], nargs=2, type=int, help='Input images will be resized to this for data argumentation.')

'''Configure'''
parser.add_argument('--network', type=str, default='vggnet_localization')
parser.add_argument('--model_dir', type=str, default='/notebooks/global_localization/dual_resnet_torch', help='rnn, gru, or lstm')


parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/train_dense'])
'''
#parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/test'])
parser.add_argument('--train_dataset', type=str, default = ['/notebooks/michigan_nn_data/2012_01_08'])
'''
parser.add_argument('--norm_tensor', type=str, default = ['/notebooks/global_localization/norm_mean_std.pt'])

parser.add_argument('--seed', default=1337, type=int)
parser.add_argument('--save_every', type=int, default=3000, help='save frequency')
parser.add_argument('--display', type=int, default=20, help='display frequency')
parser.add_argument('--tensorboard', type=bool, default=True, help='open tensorboard')
parser.add_argument('--cuda_device', type=int, default=1, help='cuda device')

sys.argv = ['']
args = parser.parse_args()

# Load Dataset

In [3]:
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchvision.transforms as transforms
import tf.transformations as tf_tran
from tqdm import tqdm
from PIL import Image
import numpy as np
import random

import gpytorch

import torch.nn as nn
import torch.optim as optim
from torchlib import resnet, vggnet
from torchlib.cnn_auxiliary import normalize, denormalize_navie, denormalize, get_relative_pose, translational_rotational_loss
from torchlib.utils import LocalizationDataset, display_loss, data2tensorboard
import time

path = args.train_dataset[0]
x_dense = torch.load(os.path.join(path,'x.pt'))
y_dense = torch.load(os.path.join(path,'y.pt'))
dataset = TensorDataset(x_dense, y_dense)

[args.norm_mean, args.norm_std] = torch.load(*args.norm_tensor)
print('Load norm and std:',*args.norm_tensor)

dataloader = DataLoader(dataset, batch_size=args.batch_size, \
                        shuffle=True, num_workers=0, \
                        drop_last=True, pin_memory=True)

Load norm and std: /notebooks/global_localization/norm_mean_std.pt


# Define Model

In [4]:
from torch.cuda.amp import autocast, GradScaler
from torchlib.GPs import Backbone, NN, BaseModule

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.global_context = vggnet.vggnet(input_channel=2048,opt="context")
        self.global_regressor = vggnet.vggnet(opt="regressor")
        
    def forward(self,input_data):
        #context_feat = self.global_context(input_data)
        context_feat = input_data
        output,feature_t, feature_r = self.global_regressor(context_feat)
        return output, feature_t, feature_r
        
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = Backbone()
        self.nn = NN()
        
    def forward(self, input_data):
        #dense_feat = self.backbone(input_data)
        dense_feat = input_data
        output, _, _ = self.nn(dense_feat)
        trans_pred, rot_pred = torch.split(output, [3, 4], dim=1)
        return trans_pred, rot_pred
        
class PosePredictor(BaseModule):
    def __init__(self, norm_mean, norm_std, args, is_training=True, mixed_precision=True):
        super().__init__(norm_mean, norm_std, args)
        self.model = Model().to(self.device)
        
        self.disable_requires_grad(self.model.backbone)
        self.disable_requires_grad(self.model.nn.global_context)
        
        self.mixed_precision = mixed_precision
        if self.mixed_precision:
            self.scaler = GradScaler()
        if is_training:
            self.optimizer = optim.Adam(self.model.parameters(), 
                                        lr=args.learning_rate, 
                                        weight_decay=args.weight_decay)
            self.scheduler = optim.lr_scheduler.LambdaLR(optimizer=self.optimizer,
                                                             lr_lambda=lambda epoch: args.decay_rate**epoch)
        else:
            self.disable_requires_grad(self.model)
    
    def train(self, x, y):
        # Step 0: zero grad
        self.optimizer.zero_grad()
        
        start = time.time()
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        if args.is_normalization:
            y = normalize(y,self.norm_mean, self.norm_std)
            
        # Step 2: training
        assert trainer.model.training == True
        if self.mixed_precision:
            with autocast():
                single_loss = self._loss(x, y)
            self.scaler.scale(single_loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            single_loss = self._loss(x, y)
            single_loss.backward()
            self.optimizer.step()
            
        batch_time = time.time() - start
        
        return float(single_loss), batch_time
    
    def _loss(self, x, y):

        trans_pred, rot_pred = self.model(x)
        trans_gt, rot_gt = torch.split(y, [3, 4], dim=1)
    
        trans_loss = nn.functional.mse_loss(input=trans_pred, target=trans_gt)
        trans_loss += torch.sum((trans_pred - trans_gt)**2,dim=1).mean()
        
        rot_loss = 1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_gt),dim=1)))

        loss = trans_loss + args.lamda_weights * rot_loss
    
        return loss
    
    def eval_forward(self,x,y,output_denormalize = True):
        # Step 1: get data
        x,y = x.to(self.device),y.to(self.device)
        
        # Step 2: forward
        assert trainer.model.training == False
        
        trans_pred, rot_pred = self.model(x)

        if args.is_normalization and output_denormalize:
            trans_pred = denormalize_navie(trans_pred, self.norm_mean, self.norm_std)
            
        # Step 3: split output
        trans_gt, rot_gt = torch.split(y, [3, 4], dim=1)
        return trans_pred, rot_pred, trans_gt, rot_gt

In [5]:
trainer = PosePredictor(args.norm_mean,args.norm_std,args,mixed_precision=True)
'''
state_dict = torch.load(os.path.join(trainer.args.model_dir, 'pretrained.pth'))
for key in list(state_dict):
    if 'resnet' not in key:
        state_dict.pop(key)
trainer.model.backbone.load_state_dict(state_dict)

state_dict = torch.load(os.path.join(trainer.args.model_dir, 'pretrained.pth'))
for key in list(state_dict):
    if 'resnet' in key:
        state_dict.pop(key)
trainer.model.nn.load_state_dict(state_dict)
'''
trainer.load_model('pretrained_feature.pth')
#trainer.load_model('pretrained_old.pth',strict = False)
trainer.show_require_grad()

Successfully loaded model to TITAN Xp.
nn.global_regressor.regressor.fc1_trans.0.weight torch.Size([4096, 6400])
nn.global_regressor.regressor.fc1_trans.0.bias torch.Size([4096])
nn.global_regressor.regressor.fc2_trans.0.weight torch.Size([4096, 4096])
nn.global_regressor.regressor.fc2_trans.0.bias torch.Size([4096])
nn.global_regressor.regressor.fc3_trans.0.weight torch.Size([128, 4096])
nn.global_regressor.regressor.fc3_trans.0.bias torch.Size([128])
nn.global_regressor.regressor.logits_t.weight torch.Size([3, 128])
nn.global_regressor.regressor.logits_t.bias torch.Size([3])
nn.global_regressor.regressor.fc1_rot.0.weight torch.Size([4096, 6400])
nn.global_regressor.regressor.fc1_rot.0.bias torch.Size([4096])
nn.global_regressor.regressor.fc2_rot.0.weight torch.Size([4096, 4096])
nn.global_regressor.regressor.fc2_rot.0.bias torch.Size([4096])
nn.global_regressor.regressor.fc3_rot.0.weight torch.Size([128, 4096])
nn.global_regressor.regressor.fc3_rot.0.bias torch.Size([128])
nn.global_

# Training

In [6]:
if args.tensorboard:
    import os
    os.system('rm -rf runs/nn')
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter('runs/nn')
    time.sleep(3)

In [7]:
trainer.model.train()
for e in range(args.num_epochs):
    minibatch_iter = tqdm(dataloader)
    train_loss = 0.
    for b, data in enumerate(minibatch_iter):
        x,y = data

        loss, batch_time = trainer.train(x,y)
        
        train_loss += loss
        ave_loss = train_loss/(b+1)
        step = e*len(dataloader)+(b+1)
        # display data
        minibatch_iter.set_postfix(ave = ave_loss, loss=loss,lr=trainer.scheduler.get_last_lr()[0])
        # tensorboard
        trainer.data2tensorboard(writer,'training loss',{'item loss':loss,'batch loss':ave_loss},step)
        # save model
        #trainer.save_model_step(e,step)
        # step scheduler
        trainer.schedule_step(step,102)

100%|██████████| 51/51 [00:19<00:00,  2.65it/s, ave=0.00132, loss=0.00136, lr=8.48e-6]


100%|██████████| 51/51 [00:19<00:00,  2.65it/s, ave=0.0013, loss=0.00135, lr=6.23e-6] 


In [8]:
trainer.save_model('pretrained_nn.pth')

Saving model to pretrained_nn.pth


# Evaluation

In [9]:
trainer.model.eval()
for e in range(args.num_epochs):
    minibatch_iter = tqdm(dataloader)
    train_loss = 0.
    for b, data in enumerate(minibatch_iter):
        x,y = data
        break
    break

trans_pred, rot_pred, trans_gt, rot_gt = trainer.eval_forward(x,y)

  0%|          | 0/51 [00:00<?, ?it/s]


In [10]:
trans_pred

tensor([[-123.0586,  346.9702,   -9.8569],
        [ -77.4997,  484.1308,  -10.9497],
        [-297.4271,  551.9941,  -12.5403],
        ...,
        [-109.6411,  598.9783,  -12.1633],
        [-122.5284,  703.4795,  -12.1626],
        [-193.5782,  415.9282,  -13.1029]], device='cuda:1',
       grad_fn=<AddBackward0>)

In [11]:
trans_gt

tensor([[-122.8274,  344.5215,   -9.9254],
        [ -76.6735,  483.1304,  -10.9046],
        [-300.2071,  551.7280,  -12.5239],
        ...,
        [-108.3895,  597.5125,  -12.0480],
        [-123.8316,  700.7715,  -12.1452],
        [-196.0222,  413.3089,  -13.3092]], device='cuda:1')

new: 7-21  0.0019

new: 5-15  0.0013

old: 8-25  0.0021

In [12]:
nn.functional.mse_loss(input=trans_pred, target=trans_gt)

tensor(5.0974, device='cuda:1', grad_fn=<MseLossBackward>)

In [13]:
torch.sum((trans_pred - trans_gt)**2,dim=1).mean()

tensor(15.2922, device='cuda:1', grad_fn=<MeanBackward0>)

In [15]:
rot_pred

tensor([[ 0.0588, -0.9982, -0.0040,  0.0064],
        [ 0.9976, -0.0666, -0.0109,  0.0124],
        [ 0.7255, -0.6882, -0.0105,  0.0054],
        ...,
        [-0.5246, -0.8513, -0.0087, -0.0071],
        [-0.0168, -0.9998, -0.0047,  0.0052],
        [ 0.1972, -0.9803, -0.0079,  0.0051]], device='cuda:1',
       grad_fn=<SplitWithSizesBackward>)

In [16]:
rot_gt

tensor([[ 0.0212,  0.9995,  0.0234, -0.0078],
        [ 0.9943, -0.1052, -0.0080,  0.0133],
        [ 0.7271, -0.6864,  0.0058, -0.0059],
        ...,
        [ 0.4042,  0.9140,  0.0256,  0.0222],
        [-0.1501,  0.9870,  0.0349,  0.0464],
        [-0.2323,  0.9722, -0.0104, -0.0263]], device='cuda:1')

In [18]:
1. - torch.mean(torch.square(torch.sum(torch.mul(rot_pred,rot_gt),dim=1)))

tensor(0.0289, device='cuda:1', grad_fn=<RsubBackward1>)